<a href="https://colab.research.google.com/github/DrashiDave/DATA-602/blob/main/Week%20-%2010/Week_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1 - Hyperparameter optimization.

The template provides a dataset that derives from the merged weather-citation dataframe that you generated
    	in Week 2.  Each observation in the data frame represents the number of red light citations issued each hour since 2020, together with
    	a bad weather indicator for that hour (refer to the Week 2 solution).  The dataframe includes additional
    	features such as circular components of the day of week, time of day, and day of year and indicator variables indicating whether the day is a weekend or U.S. federal holiday.
    	The template provides code to split into training and test instances, and to scale both.

Using a regression algorithm of your choice to predict the number of citations from the other features:

1. Using a hyperparameter optimization method of your choice, find optimal hyperparameter values that maximize the regressor's $R^2$ value.
2. Print the selected hyperparameter values.
3. Fit the estimator, using the optimal hyperparameters that you identified in (a), to the full set of training data
4. Evaluate the $R^2$ metric against the provided test data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


The below code loads and prepares the test and training data sets.

In [ ]:
df = pd.read_parquet("/content/drive/Shareddrives/DS602-F22/Data/weather_week10.parquet")
y = df.pop("count")
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
ct = ColumnTransformer(
    [
        ("ts", MinMaxScaler(), ["timestamp"]),
        ("drop", "drop", ["wobsts"])
    ], remainder="passthrough"
)
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)
y_train = y_train.values
y_test = y_test.values

Defining the hyperparameter grid that RandomizedSearchCV will explore for optimizing the RandomForestRegressor. This grid includes options for tree depth, number of estimators, sample split criteria, and whether or not to bootstrap samples.

In [ ]:
# Defining the Hyperparameter Grid

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True]
}

Initializing the RandomForestRegressor and configuring RandomizedSearchCV to try 50 random combinations of hyperparameters from the defined grid, using cross-validation to maximize the 𝑅2 score.

In [ ]:
# Initializing and Running RandomizedSearchCV
rf = RandomForestRegressor(random_state=42)
random_search = RandomizedSearchCV(
    estimator = rf,
    param_distributions = param_grid,
    n_iter = 25,
    scoring = 'r2',
    cv = 3,
    verbose = 2,
    random_state = 42,
    n_jobs = -1,
    error_score='raise'
)
random_search.fit(X_train, y_train)
print("Best hyperparameters:", random_search.best_params_)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best hyperparameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 20, 'bootstrap': True}


Using the best parameters found by RandomizedSearchCV, retraining the RandomForestRegressor on the full training set to create the final model.

In [ ]:
best_rf = random_search.best_estimator_
best_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=20, max_features='log2', min_samples_split=5,
                      n_estimators=200, random_state=42)

Evaluating the model’s performance by calculating the 𝑅2 score on the test set to measure how well the model predicts citation counts based on the weather and temporal features.

In [ ]:
test_r2 = best_rf.score(X_test,y_test)
print("Test R^2 Score:", test_r2)

Test R^2 Score: 0.8248449983587935


# Problem 2 - Evaluation
This problem is an extension of the problem sets 6 and 7.  Refer to the Model Evaluation and Selection section from Week 8 for guidance on model evaluation.

After completing problem set 7, you should have 5 models: the 3 classifiers from week 7 problem 1, the voting model from week 7, problem 2, and the stacking model from week 7, problem 3.  If you wish, you may also include the SVM model you developed for week 6, problem 2.  Then:

The below code performs the required preprocessing from Week 7:

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

#fetch OpenML data
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False,
                    parser="auto")

#split into test/training sets
N=60000
X_train, y_train = X[:N, :], y[:N]
X_test, y_test = X[N:, :], y[N:]

#scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# PCA
pca = PCA(n_components = 0.75)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# recoded target variable
recode_fn = lambda y: np.choose(np.isin(y, list("01234")), [-1,1])
y_test, y_train = (recode_fn(y) for y in [y_test, y_train])



You may use the following models if you wish, or use the classifiers you developed in the homework assignments.

In [ ]:
non_ensemble_classifiers = {'svm': SVC(coef0=0.3864834922270453, degree=4, kernel='poly'),
               'xgboost': XGBClassifier(learning_rate=0.19299343762455148, max_depth=9, n_estimators=409),
               'rfc': RandomForestClassifier(max_depth=18, max_features=0.26213576122784343,
                        min_samples_leaf=6),
               'logreg': Pipeline(
                   steps=[
                          ('polynomialfeatures', PolynomialFeatures()),
                          ('logisticregression',
                            LogisticRegression(C=0.00013211684329604987, max_iter=2000))
                          ]),
                }
ensemble_classifiers = {
    'voting': VotingClassifier([(k, v) for k, v in non_ensemble_classifiers.items()]),
    'stacking': StackingClassifier([(k, v) for k, v in non_ensemble_classifiers.items()])
}






Then:

a. Select a model from the candidate models.  You may, but are not required to, select the best-performing model.  Justify your rationale for selecting that model.  You may refer back to your solution and the posted solution from prior assignments to inform your selection.  Document any assumptions.



**Model Selection:**

* I selected the SVM model for evaluation as it achieved the best F1 score in Week 7 among the tested models (Random Forest, Gradient Boosting, Voting Ensemble, Stacking Ensemble, and SVM).
* This suggests that the SVM model captures the underlying patterns in the data well and performs consistently across classes, which is essential for balanced accuracy on this dataset.
* This indicates that the SVM was the most effective at capturing class-specific characteristics, making it a suitable candidate for further evaluation.


b. Train the selected model on the full set of training data.  (As with earlier assignment, you may train on a subset of data if training takes too long.)


In [ ]:
# Training the SVM model

svm_model = SVC(coef0=0.3864834922270453, degree=4, kernel='poly', random_state=42)
svm_model.fit(X_train, y_train)

SVC(coef0=0.3864834922270453, degree=4, kernel='poly', random_state=42)

c. Preprocess the test data (last 10,000 observations of MNIST) identically to the training data.


We already preprocessed the test data identically to the training data, using StandardScaler for normalization and PCA for dimensionality reduction to 75% explained variance.
This ensures consistency across training and test sets, allowing for accurate evaluation.

In [ ]:
X_test[:1, :]

array([[-5.03127351,  3.64112918, -7.32268689,  0.54986598, -2.77308791,
         2.10817192,  3.76234458,  6.8195386 , -4.07925865,  2.28805112,
        -0.77496835,  4.03829138, -1.68804222, -1.24066622, -1.05615137,
         0.88068608, -1.61994675, -2.97384063,  1.71794568, -2.84879893,
         0.94924206, -0.03903973,  1.97077762,  0.32725795,  4.16806047,
        -0.60561678, -1.44824394,  1.16477363,  0.36972318, -0.47856909,
         2.65222156, -1.58789162,  0.18472617,  0.64628324,  0.37560366,
         0.46404012, -1.21253202,  2.14328094,  2.7007946 , -1.69455615,
        -2.1579641 , -0.20540823,  1.44556758, -0.59223814,  0.85916959,
        -0.9053983 , -0.19027172,  0.16669902,  0.20493263, -0.45902117,
         1.63698415, -1.21857818, -1.65696904, -1.46428435, -0.69669855,
        -0.76318435, -0.52334722, -0.42755947, -0.04484432, -0.82979789,
        -0.46332114, -0.67451862, -1.49102635,  0.06895393,  0.60481114,
         0.28821297, -0.52112234, -0.45263761, -0.6

In [ ]:
y_test[1]

1

d. Use the fully-trained selected model from (b) to evaluate performance of the preprocessed test data from (c).  Confirm that
the accuracy of the test model is comparable to the expected performance of the model prior to selection.



In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Evaluating the SVM model on the test data
y_pred = svm_model.predict(X_test)

# Calculating performance metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Test Accuracy:", accuracy)
print("Test F1 Score:", f1)

Test Accuracy: 0.9802
Test F1 Score: 0.9807130333138515


**Model Evaluation Summary**

The SVM model achieved a Test Accuracy of 98.02% and a Test F1 Score of 0.9807 on the preprocessed MNIST test data. These results demonstrate that the model is highly effective in distinguishing between the recoded classes. The accuracy and F1 score align with the model’s performance during training, confirming its consistency and robustness on unseen data.